In [ ]:
import os
import torch
import torch.nn.functional as F
import tiktoken
from model import GPT

encoding = tiktoken.get_encoding("gpt2")
vocab_length = 50257
sequence_length = 128
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#Generate Fnc
def generate(model, seed=[50256],tempereture=1):
    model.eval()

    while True:
        #seed.insert(0, 49960)
        seed.append(49960)
        if len(seed)==sequence_length:
          break
    # Convert to tensor and add batch dimension.
    generated = torch.tensor(seed, dtype=torch.long, device=device).unsqueeze(0)

    while True:
        with torch.no_grad():
            # Always feed the last 128 tokens.
            input_tokens = generated[:, -sequence_length:]
            logits = model(input_tokens)
            # Reshape logits to [batch, sequence_length, vocab_length]
            logits = logits.view(input_tokens.size(0), sequence_length, vocab_length)
            # Use the logits for the last token.
            probs = F.softmax(logits[:, -1, :]/tempereture, dim=-1)
            next_token = torch.multinomial(probs, 1).item()
            if next_token == 50256:
                break
            print(encoding.decode([next_token]),end='')
        # Append the predicted token.
        generated = torch.cat([generated, torch.tensor([[next_token]], device=device)], dim=1)

# Load model and weights
model = GPT().to(device)
weight_path = f'saved_weights27.pth'
if os.path.exists(weight_path):
    model.load_state_dict(torch.load(weight_path,map_location=torch.device('cpu')))
else:
    print("Warning: No pre-trained weights found!")

In [ ]:
encoding = tiktoken.get_encoding("gpt2")
prompt=f'''The latest developments for newly opened a new work, which would be comfortable in the development of the field.'''
print(prompt,end='')
prompt=encoding.encode(prompt)
generate(model,prompt,0.5)